# Predict Workout Model Training

Predict Workout Day and distance

In [ ]:
# import loadCorrelatedData function, and pandas
from utils import loadCorrelatedData, pd

from sklearn import preprocessing
import pickle

In [ ]:
data = loadCorrelatedData()
data.head()

### Feature Extraction

In [ ]:
data['day'] = list(map(lambda t : t.day, data['Activity Date']))
data['hour'] = list(map(lambda t : t.hour, data['Activity Date']))
data['dayOfWeek'] = list(map(lambda t : t.dayofweek, data['Activity Date']))
data['isWeekend'] = ((pd.DatetimeIndex(data['Activity Date']).dayofweek) // 5 == 1).astype(int)


# convert all ride types to numeric value for decision making
le_activity = preprocessing.LabelEncoder()
le_activity.fit(data['Activity Type'])
data['rideType'] = le_activity.transform(data['Activity Type'])


# Transform weather conditions data to numeric value for decision making
le = preprocessing.LabelEncoder()
le.fit(data['weatherDesc'])
data['weather'] = le.transform(data['weatherDesc'])

features: **['hour','dayOfWeek','isWeekend','temp','wind','weather']**

In [ ]:
# Select features as list of array
X = data[['hour','dayOfWeek','isWeekend','temp','wind','weather']]
X = X.to_numpy()
X

### Prepare Predict

In [ ]:
# Set Distance values
Y_distance = data['Distance']
Y_distance = Y_distance.to_numpy()

# Set Ride Type Values
Y_rideType = data['rideType']
Y_rideType = Y_rideType.to_numpy()

## Linear Regression for Distance prediction

In [ ]:
# example of training a final regression model
from sklearn.linear_model import LinearRegression
# fit final model
model = LinearRegression()
model.fit(X[0:160], Y_distance[0:160])
# make a prediction
Xnew = X[160:167]
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
    j = 160
    print("X=%s, Predicted=%s, Actual Distance=%s, Actual Ride Type=%s" % (Xnew[i], ynew[i],Y_distance[j+i],Y_rideType[j+i]))

## Logistic Regression for RideType Prediction

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X, Y_rideType)
result_ridetype = clf.predict([[8,6,1,20,3,0]])
print("Result type prediction=%s" % result_ridetype)

result_ridetype = clf.predict([[8,6,1,10,12,1]])
print("Result type prediction=%s" % result_ridetype)

In [ ]:
#Ride for the weekend
result_ridetype = clf.predict([[8,6,1,8,12,3],[9,7,1,20,17,3],[8,2,0,14,17,3]])
print("Result type prediction=%s" % result_ridetype)

#Ride for Weekend
result = model.predict([[9,6,1,10,12,3],[9,7,1,20,17,3],[8,2,0,14,17,3]])
print(result)

### Export Model as a file

In [ ]:
# Save to file in the model folder
distance_model_file = "../web/model/distance_model_1.pkl"
with open(distance_model_file, 'wb') as file:
   pickle.dump(model, file)
   
ridetype_model_file = "../web/model/ridetype_model_1.pkl"
with open(ridetype_model_file, 'wb') as file:
   pickle.dump(clf, file)